#LABELING

annotation tool
1. CVAT : intel 에서 만든 것
2. LabelMe : MIT 에서 만든 것

https://heartbeat.fritz.ai/4-important-computer-vision-annotation-tools-you-need-to-know-in-2020-9f964931ed7

In [1]:
!pip install labelme


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for labelme: filename=labelme-4.5.9-py3-none-any.whl size=1465417 sha256=5fc96454c461057b1ae87aaea29f4e12bf531b2a81ebc6b72e1a191b814e01de
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\1a\69\e0\8e8530643e291ff5b3eeb05568d873ad42b08c15166885a288
  Created wheel for imgviz: filename=imgviz-1.2.6-py3-none-any.whl size=7674073 sha256=4d28a19ad1dad711b29e706aa87303d1fa65f7f02792a0e509c35d9f2bbe95a7
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\e6\64\f9\a28eca2133ece5f072f51282577f2f9b7d6d0492eb3d2104dd
Successfully built labelme imgviz
  Attempting uninstall: matplotlib
    Found existing installation: matpl


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'c:\\programdata\\anaconda3\\Lib\\site-packages\\pylab.py'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


#YOLOV3 코드

https://bong-sik.tistory.com/16 : 참고링크-한글 \
https://pysource.com/2019/06/27/yolo-object-detection-using-opencv-with-python/ : 참고링크-영어

https://pjreddie.com/darknet/yolo/ : 다운로드 경로 \
(YOLOv3-tiny > YOLOv3-320 > YOLOv3-416 > YOLOv3-608 = YOLOv3-spp) : 속도빠르면 정확도 떨어짐

In [26]:
import cv2 #open cv
import numpy as np

In [27]:
# Load Yolo
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [28]:
# 이미지 가져오기
img = cv2.imread("sample.jpg") #한국어? 오류남
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

In [29]:
# Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

In [30]:
# 정보를 화면에 표시
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # 좌표
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [31]:
#noise 제거
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [32]:
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()